# GoogleNet/Inception v3 (Transfer Learning)

In [6]:
import torch
import torchvision.models as models
from torch import nn, optim
from torchsummary import summary
from ptflops import get_model_complexity_info
import pandas as pd
import time

# Import custom utility module
from utils import get_transform, prepare_train_test_data, calculate_evaluation_metrics

In [7]:
# Device configuration (Select CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n Device : {device}")


 Device : cuda


### Load and prepare the data

In [8]:
# Transform data
image_input_size = (3, 299, 299) # Image input size for the model
transform = get_transform(resize_image_size = (299, 299))

# Prepare Train/Val/Test Data
train_loader, val_loader, test_loader = prepare_train_test_data(transform)

# Model keyword
model_keyword = "inceptionv3"

### Load pre-trained ResNet50 model and modify it

In [9]:
# Load pre-trained GoogleNet
# model = models.googlenet(pretrained=True) # Loading Inception v1 by default. 
model = models.inception_v3(pretrained=True) # Select specific model

# Modify the final fully connected layer for binary classification
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # Binary classification: Internal waves (1) or No waves (0)
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.1) # Implement a learning rate scheduler to adjust the learning rate during training

# Print summary
summary(model, input_size=image_input_size)

print(f"Total number of layers: {sum(1 for _ in model.modules())}")

C:\Users\ishar\.virtualenvs\ML_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ishar\.virtualenvs\ML_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 149, 149]             864
       BatchNorm2d-2         [-1, 32, 149, 149]              64
       BasicConv2d-3         [-1, 32, 149, 149]               0
            Conv2d-4         [-1, 32, 147, 147]           9,216
       BatchNorm2d-5         [-1, 32, 147, 147]              64
       BasicConv2d-6         [-1, 32, 147, 147]               0
            Conv2d-7         [-1, 64, 147, 147]          18,432
       BatchNorm2d-8         [-1, 64, 147, 147]             128
       BasicConv2d-9         [-1, 64, 147, 147]               0
        MaxPool2d-10           [-1, 64, 73, 73]               0
           Conv2d-11           [-1, 80, 73, 73]           5,120
      BatchNorm2d-12           [-1, 80, 73, 73]             160
      BasicConv2d-13           [-1, 80, 73, 73]               0
           Conv2d-14          [-1, 192,

### Train the model

In [10]:
# Initialize a df to log training/validation metrics
log_df = []

# Record the start time
start_time = time.time()

# Training loop
num_epochs = 50
best_val_loss = float('inf')  # Initialize best validation loss for saving model

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)

        # Handle InceptionOutputs
        if isinstance(outputs, tuple):  # Outputs: (logits, aux_logits)
            logits, aux_logits = outputs
        else:
            logits = outputs

        # Compute the loss using main logits
        loss = criterion(logits, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            # Handle InceptionOutputs during validation
            if isinstance(outputs, tuple):  # Outputs: (logits, aux_logits)
                logits, aux_logits = outputs
            else:
                logits = outputs

            # Compute validation loss using main logits
            loss = criterion(logits, labels)
            val_loss += loss.item()

            _, predicted = torch.max(logits, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = val_correct / val_total * 100
    
    # Adjust learning rate
    scheduler.step(val_loss)

    # Calculate FLOPs after each epoch
    with torch.no_grad():
        flops, _ = get_model_complexity_info(model, image_input_size, as_strings=False, print_per_layer_stat=False)

    # Log metrics into the DataFrame
    new_row = {
        "Epoch": epoch + 1,
        "Train_Loss": train_loss / len(train_loader),
        "Validation_Loss": val_loss / len(val_loader),
        "Validation_Accuracy": val_accuracy,
        "FLOPs": flops
    }
    log_df.append(new_row)

    # Print metrics for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss / len(train_loader):.4f}, "
          f"Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {val_accuracy:.2f}%, "
          f"GFLOPs: {(flops / 1e9 ):.2f}")

    # Save the best model based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f"model_outputs_data/best_{model_keyword}_model.pth")

# Calculate the total training time in seconds
print(f"\nTotal Training Time: {(time.time() - start_time):.2f} seconds")

# Save the DataFrame to a CSV file for later use
log_df = pd.DataFrame(log_df)
log_df.to_csv(f"model_outputs_data/model_evaluation_logs/training_logs_{model_keyword}.csv", index=False)

Epoch [1/50], Train Loss: 0.4814, Val Loss: 0.7766, Val Accuracy: 70.63%, GFLOPs: 5.75
Epoch [2/50], Train Loss: 0.3553, Val Loss: 0.3994, Val Accuracy: 79.88%, GFLOPs: 5.75
Epoch [3/50], Train Loss: 0.3071, Val Loss: 0.2891, Val Accuracy: 88.11%, GFLOPs: 5.75
Epoch [4/50], Train Loss: 0.2567, Val Loss: 1.2500, Val Accuracy: 86.64%, GFLOPs: 5.75
Epoch [5/50], Train Loss: 0.2682, Val Loss: 0.4394, Val Accuracy: 84.88%, GFLOPs: 5.75
Epoch [6/50], Train Loss: 0.2288, Val Loss: 0.2998, Val Accuracy: 89.57%, GFLOPs: 5.75
Epoch [7/50], Train Loss: 0.2065, Val Loss: 0.3425, Val Accuracy: 86.34%, GFLOPs: 5.75
Epoch [8/50], Train Loss: 0.1464, Val Loss: 0.2665, Val Accuracy: 90.75%, GFLOPs: 5.75
Epoch [9/50], Train Loss: 0.1253, Val Loss: 0.3755, Val Accuracy: 88.11%, GFLOPs: 5.75
Epoch [10/50], Train Loss: 0.1260, Val Loss: 0.3234, Val Accuracy: 89.43%, GFLOPs: 5.75
Epoch [11/50], Train Loss: 0.0993, Val Loss: 0.2916, Val Accuracy: 89.43%, GFLOPs: 5.75
Epoch [12/50], Train Loss: 0.0644, Val Lo

### Prediction and Model evaluation

In [11]:
model.eval()
# Load the best model for testing
# model.load_state_dict(torch.load(f"model_outputs_data/best_{model_keyword}_model.pth"))

# Test the model
all_labels = []
all_preds = []
all_probs = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())
        all_probs.extend(probabilities[:, 1].cpu().numpy())

# Print evaluation metrics
calculate_evaluation_metrics(all_labels, all_preds, all_probs, model_keyword)


Test Accuracy: 93.76%
Precision: 0.96
Recall: 0.92
F1-Score: 0.94

Classification Report:
                   precision    recall  f1-score   support

No Internal Waves       0.92      0.96      0.94       719
   Internal Waves       0.96      0.92      0.94       740

         accuracy                           0.94      1459
        macro avg       0.94      0.94      0.94      1459
     weighted avg       0.94      0.94      0.94      1459


Confusion Matrix:
[[689  30]
 [ 61 679]]

 Predictions saved to model_outputs_data/model_prediction_logs/inceptionv3_labels_predictions.csv
